# 🚀 PiEdge EduKit - Quick Run & Sanity Check

## What you'll learn today

* Train a tiny image classifier in PyTorch
* Export the model to **ONNX** (a portable format for deployment)
* Measure inference latency and interpret P50/P95
* (Try to) quantize to INT8 and understand why it may fail
* Evaluate the model and record a reproducible "receipt"

## Why this matters

Most real projects train in Python but deploy elsewhere (C++, mobile, web, embedded). ONNX lets us move models **out of Python** without rewriting the model by hand.

## How to use this notebook

This is a **smoke test**: it runs the whole pipeline end-to-end so your environment is correct. For learning and coding tasks, continue with **`01_training_and_export.ipynb`** → **`04_evaluate_and_verify.ipynb`**.

---

## ONNX 101

**What is ONNX?**
ONNX (Open Neural Network Exchange) is an **open standard** for representing ML models as a graph of operators (Conv, Relu, MatMul…). Many frameworks can **export** to ONNX (PyTorch, TensorFlow) and many runtimes can **execute** ONNX (ONNX Runtime, TensorRT, CoreML Tools).

**Why ONNX?**

* **Portability**: train in Python, deploy in C++/C#/Java/JS, mobile or edge.
* **Performance**: runtimes fuse ops and call optimized backends (MKL, cuDNN).
* **Interoperability**: one model file can run across platforms with different "Execution Providers" (CPU, CUDA, DirectML, NNAPI…).

**Key terms**

* **Opset**: version of the operator set supported by runtimes. We export with a specific opset (e.g., 17).
* **Static vs dynamic shapes**: fixed sizes are simpler/faster; dynamic adds flexibility.
* **Execution Provider (EP)**: the backend used by ONNX Runtime (e.g., `CPUExecutionProvider`).
* **Pre/Post-processing**: steps around the model (resize, normalize, label mapping). These **aren't** part of the ONNX graph; the app must do the same steps.


## 1️⃣ Setup & Verifiering

Först kontrollerar vi att miljön är korrekt:


In [ ]:
# Quiet noisy ORT quantizer log line (appears even with correct preprocessing)
import logging
for name in ("", "onnxruntime", "onnxruntime.quantization"):
    logging.getLogger(name).setLevel(logging.ERROR)


In [1]:
# Make notebook run from repo root (not labs/) + quiet mode
import os, sys, warnings, pathlib

# If opened from labs/, change working directory to repo root
nb_dir = pathlib.Path.cwd()
if nb_dir.name == "labs":
    os.chdir(nb_dir.parent)
    print("-> Changed working dir to repo root:", os.getcwd())

# Ensure repo root is importable
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

# Quiet progress bars and some noisy warnings
os.environ.setdefault("TQDM_DISABLE", "1")  # hide tqdm progress bars
os.environ.setdefault("PYTHONWARNINGS", "ignore")
os.environ.setdefault("ORT_LOG_SEVERITY_LEVEL", "3")  # ORT info/warn -> quiet
warnings.filterwarnings("ignore", category=UserWarning, module="onnxruntime")



-> Changed working dir to repo root: C:\Users\olabl\Documents\GitHub\piedge_edukit


In [2]:
# ruff: noqa: E401
# Cross-platform runner + live clock (no shell redirection needed)
import sys
import subprocess
import time
import threading
import shutil
from contextlib import contextmanager
from IPython.display import display

try:
    import ipywidgets as widgets
    _HAVE_WIDGETS = True
except Exception:
    _HAVE_WIDGETS = False

@contextmanager
def running_timer(label="Running…"):
    start = time.time()
    symbols = ["🕐","🕑","🕒","🕓","🕔","🕕","🕖","🕗","🕘","🕙","🕚","🕛"]
    stop = False

    if _HAVE_WIDGETS:
        w = widgets.HTML()
        display(w)
        def _tick():
            k = 0
            while not stop:
                w.value = f"<b>{symbols[k%12]}</b> {label} &nbsp; <code>{time.time()-start:.1f}s</code>"
                time.sleep(0.5); k += 1
        t = threading.Thread(target=_tick, daemon=True); t.start()
        try:
            yield
        finally:
            stop = True; t.join(timeout=0.2)
            w.value = f"✅ Done — <code>{time.time()-start:.1f}s</code>"
    else:
        width = shutil.get_terminal_size((80, 20)).columns
        def _tick():
            k = 0
            while not stop:
                msg = f"{symbols[k%12]} {label}  {time.time()-start:.1f}s"
                print("\r" + msg[:width].ljust(width), end="")
                time.sleep(0.5); k += 1
            print()
        t = threading.Thread(target=_tick, daemon=True); t.start()
        try:
            yield
        finally:
            stop = True; t.join(timeout=0.2)
            print(f"✅ Done — {time.time()-start:.1f}s")

def run_module(label, module, *args):
    """Run `python -m <module> <args>` cross-platform, capture output, raise on error."""
    with running_timer(label):
        cmd = [sys.executable, "-W", "ignore", "-m", module, *map(str, args)]
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        print(proc.stdout)
        if proc.returncode != 0:
            raise RuntimeError(f"{module} exited with code {proc.returncode}")

def run_script(label, path, *args):
    """Run `python <path> <args>` cross-platform, capture output, raise on error."""
    with running_timer(label):
        cmd = [sys.executable, "-W", "ignore", path, *map(str, args)]
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        print(proc.stdout)
        if proc.returncode != 0:
            raise RuntimeError(f"{path} exited with code {proc.returncode}")



In [3]:
# Miljökoll + självläkning (Python 3.12 + editable install)
import sys, os, importlib, subprocess
print(f"Python version: {sys.version}")
assert sys.version_info[:2] == (3, 12), f"Python 3.12 krävs, du har {sys.version_info[:2]}"

try:
    import piedge_edukit  # noqa: F401
    print("✅ PiEdge EduKit package OK")
except ModuleNotFoundError:
    # Hitta repo-roten: om vi står i labs/, gå ett steg upp
    repo_root = os.path.abspath(os.path.join(os.getcwd(), "..")) if os.path.basename(os.getcwd()) == "labs" else os.getcwd()
    print("⚠ Package saknas – installerar editable från:", repo_root)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", repo_root])
    importlib.invalidate_caches()
    import piedge_edukit  # noqa: F401
    print("✅ Package installerat")


Python version: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
✅ PiEdge EduKit package OK


In [4]:
# Paketet ska redan vara installerat av cellen ovan. Enkel sanity:
import piedge_edukit
print("✅ Paketet importeras – kör vidare!")


✅ Paketet importeras – kör vidare!


## 2️⃣ Träning & ONNX Export

Tränar en liten modell med FakeData och exporterar till ONNX:


In [5]:
# Träna modell (snabb körning för demo)
run_module("Training (FakeData)",
           "piedge_edukit.train",
           "--fakedata", "--no-pretrained",
           "--epochs", 1, "--batch-size", 256,
           "--output-dir", "./models")


HTML(value='')

Using device: cpu
[INFO] Using FakeData for training (no real images)
Preparing data...
Set 2 classes: ['class0', 'class1']
Training model with 2 classes...
Classes: ['class0', 'class1']

Epoch 1/1
Train Loss: 0.7062, Train Acc: 51.00%
Val Loss: 0.6809, Val Acc: 65.00%

Training completed! Best validation accuracy: 65.00%
Exporting model to ONNX...
[OK] Model exported to models\model.onnx (opset 17)
[OK] ONNX model verified successfully
  Input shape: (1, 3, 64, 64)
  Output shape: (1, 2)
  Output dtype: float32
[OK] Preprocessing configuration valid (hash: 9f9a96cb4a32eea9)
[OK] Labels valid: 2 classes
[OK] Model exported successfully to models\model.onnx
[OK] Training and export completed successfully!



In [6]:
# Kontrollera att modellen skapades
import os
if os.path.exists("./models/model.onnx"):
    size_mb = os.path.getsize("./models/model.onnx") / (1024*1024)
    print(f"✅ ONNX-modell skapad: {size_mb:.1f} MB")
else:
    print("❌ ONNX-modell saknas")


✅ ONNX-modell skapad: 8.5 MB


## 3️⃣ Latensbenchmark

Mäter hur snabb modellen är på CPU:


In [7]:
# Kör benchmark (snabb körning)
run_module("Benchmarking (CPU)",
           "piedge_edukit.benchmark",
           "--fakedata",
           "--model-path", "./models/model.onnx",
           "--warmup", 1, "--runs", 3,
           "--providers", "CPUExecutionProvider")


HTML(value='')

Starting latency benchmark...
Model: models\model.onnx
Data: None
Output: reports
[OK] Model loaded successfully
  Providers: ['CPUExecutionProvider']
  Input shape: ['batch_size', 3, 64, 64]
  Output shape: ['batch_size', 2]
[INFO] Generating fake test data for benchmarking
[OK] Generated 50 fake test images
Running 1 warmup iterations...
[OK] Warmup completed
Running 3 benchmark iterations...
[OK] Results saved to reports
[OK] Plot saved to reports\latency_plot.png

BENCHMARK RESULTS
Mean latency: 0.517 ms
P50 latency:  0.509 ms
P95 latency:  0.538 ms
Std deviation: 0.018 ms

[OK] Benchmark completed successfully!
Results saved to: reports



In [8]:
# Visa benchmark-resultat
if os.path.exists("./reports/latency_summary.txt"):
    with open("./reports/latency_summary.txt", "r") as f:
        print("📊 Benchmark-resultat:")
        print(f.read())
else:
    print("❌ Benchmark-rapport saknas")


📊 Benchmark-resultat:
PiEdge EduKit - Latency Benchmark Results

Version: 0.1.0
Generated: 2025-10-01 15:30:50

Version Information:
  Python: 3.12.10
  ONNX Runtime: 1.18.0
  Platform: Windows-11-10.0.26100-SP0
  Device: PC/Laptop

System Information:
  cpu_count: 20
  memory_gb: 63.39
  piedge_edukit_version: 0.1.0
  cpu_governor: N/A

Benchmark Configuration:
  Model: model.onnx
  Warmup runs: 1
  Benchmark runs: 3
  Batch size: 1

Latency Statistics (ms):
  Mean: 0.517
  Std:  0.018
  Min:  0.499
  Max:  0.541
  P50:  0.509
  P95:  0.538
  P99:  0.541



## 4️⃣ Kvantisering (INT8)

Komprimerar modellen för snabbare inference:


In [9]:
# Best Practice: Use real training images for calibration
# This ensures correct preprocessing and avoids ORT warnings
from pathlib import Path
import numpy as np
from PIL import Image

print("📊 Setting up calibration data (best practice: reuse real training images)")

# Create tiny calibration image set if data/train/ is missing
calib_dir = Path("data/train")
if not calib_dir.exists() or not any(calib_dir.rglob("*.png")):
    print("   Creating fallback calibration dataset...")
    for cls in ["class0", "class1"]:
        (calib_dir / cls).mkdir(parents=True, exist_ok=True)
        for i in range(16):  # 32 total (16 per class)
            # Synthetic but "real" PNG files
            arr = (np.random.rand(64, 64, 3) * 255).astype(np.uint8)
            Image.fromarray(arr).save(calib_dir / cls / f"sample_{i:02d}.png")
    print(f"✅ Created 32 fallback calibration images in {calib_dir}")
    print(f"   Source: Synthetic PNG files (organized like real training data)")
else:
    num_samples = sum(1 for p in calib_dir.rglob("*.png"))
    print(f"✅ Found {num_samples} existing images in {calib_dir}")
    print(f"   Source: Real training data")

print("   → Using --data-path ensures correct preprocessing (no ORT warning!)")


📊 Setting up calibration data (best practice: reuse real training images)
   Creating fallback calibration dataset...
✅ Created 32 fallback calibration images in data\train
   Source: Synthetic PNG files (organized like real training data)
   → Using --data-path ensures correct preprocessing (no ORT warning!)


In [10]:
# Kör kvantisering med RIKTIGA träningsbilder (korrekt preprocessing, ingen ORT-varning!)
try:
    if calib_dir.exists():
        # Use real training data - same preprocessing as model was trained with
        run_module("Quantization (INT8 with real calibration data)",
                   "piedge_edukit.quantization",
                   "--data-path", str(calib_dir),
                   "--model-path", "./models/model.onnx",
                   "--calib-size", 32)
    else:
        # Fallback to FakeData (will show ORT warning)
        run_module("Quantization (INT8 attempt with FakeData)",
                   "piedge_edukit.quantization",
                   "--fakedata",
                   "--model-path", "./models/model.onnx",
                   "--calib-size", 16)
except RuntimeError as e:
    print("⚠️ Quantization step failed (OK for demo):", e)


HTML(value='')

Starting quantization benchmark...
FP32 Model: models\model.onnx
Data: data\train
Output: reports
[OK] Preprocessing configuration valid (hash: 9f9a96cb4a32eea9)
[OK] Labels valid: 2 classes
Found 32 calibration images
Preprocessed 32 calibration images
Quantizing model to INT8...
[ERROR] Quantization failed: 'list' object has no attribute 'get_next'
This may be due to unsupported operations or ONNX Runtime version.
Continuing with FP32 model only.

Benchmarking FP32 model...
Benchmarking FP32 model...
Benchmarking FP32 model...

Skipping INT8 benchmarking (quantization failed)
[OK] Comparison results saved to reports
[OK] Comparison plot saved to reports\quantization_comparison.png

QUANTIZATION BENCHMARK RESULTS
Quantization Status: FAILED
Error: INT8 quantization failed - continuing with FP32 only
FP32 Latency: 0.590 ms
FP32 Size: 8.47 MB

[OK] Quantization benchmark completed successfully!
Results saved to: reports



In [11]:
# Visa kvantiseringsresultat
if os.path.exists("./reports/quantization_summary.txt"):
    with open("./reports/quantization_summary.txt", "r") as f:
        print("⚡ Kvantiseringsresultat:")
        print(f.read())
else:
    print("❌ Kvantiseringsrapport saknas")

# Tydlig notis om INT8-fail
print("\nℹ️ INT8-kvantisering kan fallera på vissa miljöer. I denna lektion är **FP32** godkänt; verify accepterar fallback.")


⚡ Kvantiseringsresultat:
PiEdge EduKit - Quantization Comparison Results

Version: 0.1.0
Generated: 2025-10-01 15:31:03

Model Comparison:
  FP32 Size: 8.47 MB
  INT8 Size: N/A (quantization failed)
  Size Reduction: N/A (quantization failed)

Latency Comparison:
  FP32 Mean: 0.590 ms
  INT8 Mean: N/A (quantization failed)
  Speedup: N/A (quantization failed)

Accuracy Comparison: N/A (quantization failed)


ℹ️ INT8-kvantisering kan fallera på vissa miljöer. I denna lektion är **FP32** godkänt; verify accepterar fallback.


## 5️⃣ Utvärdering & Verifiering

Testar modellen och genererar kvitto:


In [12]:
# Kör utvärdering
run_script("Evaluating ONNX",
           "scripts/evaluate_onnx.py",
           "--model", "./models/model.onnx",
           "--fakedata", "--limit", 16)


HTML(value='')

Wrote reports/confusion_matrix.png and reports/eval_summary.txt



In [13]:
# Kör verifiering och generera kvitto
run_script("Verifying & generating receipt", "verify.py")


HTML(value='')

PASS



In [14]:
# Visa kvitto
import json
if os.path.exists("./progress/receipt.json"):
    with open("./progress/receipt.json", "r") as f:
        receipt = json.load(f)
    print("📋 Verifieringskvitto:")
    print(f"Status: {'✅ PASS' if receipt['pass'] else '❌ FAIL'}")
    print(f"Timestamp: {receipt['timestamp']}")
    print("\nKontroller:")
    for check in receipt['checks']:
        status = "✅" if check['ok'] else "❌"
        print(f"  {status} {check['name']}: {check['reason']}")
else:
    print("❌ Kvitto saknas")


📋 Verifieringskvitto:
Status: ✅ PASS
Timestamp: 2025-10-01T13:31:11.654172Z

Kontroller:
  ✅ artifacts_exist: All required artifacts present
  ✅ latency_summary_parse: Successfully parsed latency metrics
  ✅ quantization_report_exists: Quantization report exists
  ✅ quant_speedup_or_fallback: INT8 quantization failed - fallback accepted
  ✅ evaluation_reports_exist: Evaluation reports present


## 🎉 Klar!

Du har nu kört hela PiEdge EduKit-lektionen! 

**Nästa steg**: Gå till `01_training_and_export.ipynb` för att förstå vad som hände under träningen.

**Genererade filer**:
- `models/model.onnx` - Tränad modell
- `reports/` - Benchmark och kvantiseringsrapporter
- `progress/receipt.json` - Verifieringskvitto


## Mini-Glossary

* **ONNX**: portable model format defined as an operator graph.
* **Opset**: versioned set of operators supported by runtimes.
* **Execution Provider (EP)**: backend used by ONNX Runtime (CPU, CUDA, DirectML…).
* **Latency**: time per request; **Throughput**: requests per second.
* **P50/P95/P99**: latency percentiles; tails indicate rare slow requests.
* **Quantization (PTQ)**: convert FP32 to INT8 using calibration data.
* **Calibration**: running representative samples to estimate activation ranges.
